# Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

from scipy import stats
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from statsmodels.stats.multicomp import MultiComparison
import glob

# Additional Imports
import os, json, math, time
import tmdbsimple as tmdb
from tqdm.notebook import tqdm_notebook

import warnings
warnings.filterwarnings('ignore')

# Custom Functions

In [2]:
def write_json(new_data, filename): 
    """Appends a list of records (new_data) to a json file (filename). 
    Adapted from: https://www.geeksforgeeks.org/append-to-json-file-using-python/"""  
    
    with open(filename,'r+') as file:
        # First we load existing data into a dict.
        file_data = json.load(file)
        ## Choose extend or append
        if (type(new_data) == list) & (type(file_data) == list):
            file_data.extend(new_data)
        else:
             file_data.append(new_data)
        # Sets file's current position at offset.
        file.seek(0)
        # convert back to json.
        json.dump(file_data, file)

In [3]:
def get_movie_with_rating(movie_id):
    """Copied from Coding Dojo Learning Platform"""
    # Get the movie object for the current id
    movie = tmdb.Movies(movie_id)
    # save the .info .releases dictionaries
    info = movie.info()
    releases = movie.releases()
    # Loop through countries in releases
    for c in releases['countries']:
        # if the country abbreviation==US
        if c['iso_3166_1' ] =='US':
            ## save a "certification" key in the info dict with the certification
            info['certification'] = c['certification']
    
    return info

# Load Data

## Title Basics

In [9]:
basics = pd.read_csv(r"C:\Users\Evan\Documents\Github\IMDB\Data\title_basics.csv.gz")
basics

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,NaN,118,"Comedy,Fantasy,Romance"
1,tt0043139,movie,Life of a Beijing Policeman,Wo zhe yi bei zi,0,2013.0,NaN,120,"Drama,History"
2,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020.0,NaN,70,Drama
3,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018.0,NaN,122,Drama
4,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005.0,NaN,100,"Comedy,Horror,Sci-Fi"
...,...,...,...,...,...,...,...,...,...
81824,tt9914942,movie,Life Without Sara Amat,La vida sense la Sara Amat,0,2019.0,NaN,74,Drama
81825,tt9915872,movie,The Last White Witch,My Girlfriend is a Wizard,0,2019.0,NaN,97,"Comedy,Drama,Fantasy"
81826,tt9916170,movie,The Rehearsal,O Ensaio,0,2019.0,NaN,51,Drama
81827,tt9916190,movie,Safeguard,Safeguard,0,2020.0,NaN,95,"Action,Adventure,Thriller"


# API Extraction from TMDB

## Create API Connection

In [4]:
import json
with open('/Users/Evan/.secret/tmdb_api.json', 'r') as f:
    login = json.load(f)
## Display the keys of the loaded dict
login.keys()

dict_keys(['client-id', 'api-key'])

In [5]:
tmdb.API_KEY =  login['api-key']

## Folders to store and keep existing data

In [6]:
FOLDER = 'Data/'
os.makedirs(FOLDER, exist_ok=True)
os.listdir(FOLDER)

['final_tmdb_data_2000.csv.gz',
 'final_tmdb_data_2001.csv.gz',
 'title_akas.csv.gz',
 'title_basics.csv.gz',
 'title_ratings.csv.gz',
 'tmdb_api_results_2000.json',
 'tmdb_api_results_2001.json',
 'tmdb_results_combined.csv.gz']

## Specify range of years to pull with API

In [7]:
YEARS_TO_GET = range(2010, 2023)
errors = []

In [ ]:
"""Adapted from Coding Dojo Learning Platform"""
#START OF OUTER LOOP
for YEAR in tqdm_notebook(YEARS_TO_GET, desc='YEARS', position=0):
    
    #Defining the JSON file to store results for year
    JSON_FILE = f'{FOLDER}tmdb_api_results_{YEAR}.json'

    # Check if file exists
    file_exists = os.path.isfile(JSON_FILE)
    # If it does not exist: create it
    if file_exists == False:
        # save an empty dict with just "imdb_id" to the new json file.
        with open(JSON_FILE,'w') as f:
            json.dump([{'imdb_id':0}],f)

    #Saving new year as the current df
    #Filter for basics df for only specific year
    df = basics[basics['startYear']==YEAR].copy()
    # saving movie ids to list
    movie_ids = df['tconst'].copy()
    
    ###Stop gap measure to insure previously retrieved data isn't retrieved again
    # Load existing data from json into a dataframe called "previous_df"
    previous_df = pd.read_json(JSON_FILE)

    # filter out any ids that are already in the JSON_FILE
    movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]
    
#INNER LOOP
    
    #Get index and movie id from list
    for movie_id in tqdm_notebook(movie_ids_to_get,
                                  desc=f'Movies from {YEAR}',
                                  position=1,
                                  leave=True):
        try:
            # Retrieve the data for the movie id
            temp = get_movie_with_rating(movie_id)  
            # Append/extend results to existing file using a pre-made function
            write_json(temp,JSON_FILE)
            # Short 20 ms sleep to prevent overwhelming server
            time.sleep(0.02)

        except Exception as e:
            errors.append([movie_id, e])
#END OF INNER LOOP
    
#SECOND PART OF OUTERLOOP
    
    #save file as .csv.gz  
    final_year_df = pd.read_json(JSON_FILE)
    final_year_df.to_csv(f"{FOLDER}final_tmdb_data_{YEAR}.csv.gz", 
                         compression="gzip", index=False)

# print number of errors
print(f"- Total errors: {len(errors)}")

YEARS:   0%|          | 0/13 [00:00<?, ?it/s]

Movies from 2010:   0%|          | 0/3861 [00:00<?, ?it/s]

In [ ]:
## Use Glob to Combine Files from 2010 to 2023
q = "Data/final_tmdb_data_*.csv.gz"
chunked_files = glob.glob(q)
# Showing the first 5
files_to_get = chunked_files[2:]
files_to_get

In [ ]:
## Loading all files as df and appending to a list
df_list = []
for file in files_to_get:
    temp_df = pd.read_csv(file, index_col=0)
    df_list.append(temp_df)
    
## Concatenating the list of dfs into 1 combined
df_combined = pd.concat(df_list)
df_combined

In [ ]:
df_combined.drop(index=0, inplace=True)
df_combined.drop(index='0', inplace=True)
df_combined

### Clean Groups
- Will change:
 - PG-13 with extra white space to PG-13
 - 10 to G
 - Unrated to NR

In [ ]:
df_combined['certification'].value_counts()

In [ ]:
df_combined['certification'] = df_combined['certification'].str.replace('10', 'PG')
df_combined['certification'] = df_combined['certification'].str.replace('PG-13 ', 'PG-13')
df_combined['certification'] = df_combined['certification'].str.replace('Unrated', 'NR')
df_combined['certification'].value_counts()

# Hypothesis Testing

## Question 1: Does the MPAA(G/PG/PG-13/R) rating of a movie affect how much revenue the movie generates?

### Step 1: State Null/Alternative Hypothesis

- **Null Hypothesis:** The movie rating does not affect how much revenue a movie generates


- **Alternative Hypothesis:** The movie ratings have a significant affect on how much revenue a movie generates


- **Alpha:** .05

### STEP 2: Type of Test

- **ANOVA**
 - Comparing numeric data: revenue
 - More than 2 groups: G, PG, PG-13, R

In [ ]:
df1 = df_combined.copy()

#### We should drop the movies with ratings of NR and NC-17 as were are not dealing with these ratings

In [ ]:
df1 = df1[(df1['certification'] != 'NR') & (df1['certification'] != 'NC-17')]
df1['certification'].value_counts()

### Separate into groups based on rating

In [ ]:
groups = {}
## Loop through all unique categories
for i in df1['certification'].unique():
    ## Get series for group and rename
    data = df1.loc[df1['certification']==i,'revenue'].copy()
    
    # save into the dictionary
    groups[i] = data
groups.keys()

In [ ]:
# Code from https://stackoverflow.com/questions/24068306/
# is-there-a-way-to-remove-nan-from-a-dictionary-filled-with-data
# Raghul Raj
groups = {k: groups[k] for k in groups if not pd.isna(k)}

In [ ]:
groups.keys()


### Step 3: Assumptions
- No significant outliers
- Normality
- Equal Variance

#### Outliers

In [ ]:
for i in groups:
    print(f'{i} - Number of rows: {len(groups[i])}')
    z_score = stats.zscore(groups[i])
    outliers = abs(z_score)>3
    print(f'{i} - Number of outliers: {np.sum(outliers)}')
    
    groups[i] = groups[i][~outliers]
    print(f'{i} - Number of rows: {len(groups[i])}')
    print('')

#### Normality
- The groups all have > 15 samples

In [ ]:
## Running normal test on each group and confirming there are >20 in each group
norm_results = {}
for i, data in groups.items():
    stat, p = stats.normaltest(data)
    ## save the p val, test statistic, and the size of the group
    norm_results[i] = {'n': len(data),
                             'p':p,
                             'test stat':stat,}
## convert to a dataframe
norm_results_df = pd.DataFrame(norm_results).T
norm_results_df

#### Equal Variance

In [ ]:
stats.levene(*groups.values())

### Step 4: Perform Test and Interpret Result
- Since Equal Variance assumption was no met:

 - We will run Kruskal-Wallis test
 
 
- p value < 0.05(alpha), reject Null Hypothesis and support Alternative Hypothesis that movie rating does indeed affect revenue

In [ ]:
result_q1 = stats.kruskal( *groups.values())
result_q1

In [ ]:
## is our result significant
print(f"p-value={result_q1.pvalue:.10f}")
print(f"Significant: {result_q1.pvalue <.05}")

### Visualization

In [ ]:
groups_df = pd.DataFrame(index=[0], data={'revenue': 0, 'certification':0})
groups_df

In [ ]:
groups_df.drop(index=0, inplace=True)
groups_df

In [ ]:
fig, ax = plt.subplots()

ax = sns.barplot(data=groups_df, x=groups_df['certification'], y=groups_df['revenue'])
ax.set_title('Certification by Revenue');

### Step 5: Pairwise Tukey Comparison Test
- Shows 4 significant results
 - reject column
- Reject null hypothesis for groups:
    - G and PG
    - G and PG-13
    - PG and R
    - PG-13 and R
- There is a significant difference in the results of Ratings PG and PG-13 compared to the other 2 ratings
- PG and PG-13 show the most revenue

In [ ]:
values = groups_df['revenue']
labels = groups_df['certification']

In [ ]:
tukey_q1 = pairwise_tukeyhsd(values, labels)
tukey_q1.summary()

## Question 2: Do movies that are over 2.5 hours long earn more revenue than movies that are 1.5 hours long or less

### STEP 1: State Null/Alternative Hypothesis
- Null Hypothesis: there is no difference in earned revenue between movies that are over 2.5 hours long and movies that are 1.5 hours long (or less)

- Alternative Hypothesis: there is a difference in earned revenue between movies that are over 2.5 hours long and movies that are 1.5 hours long (or less)

- Alpha: 0.05

### STEP 2: Type of Test
- Independent T-test:
 - Comparing numeric data: revenue
 - Comparing 2 samples: movies that are over 2.5 hours long and movies that are 1.5 hours (or less) long


#### Create 2 Groups of Data

In [ ]:
# long_movie
long_movie = df_combined[df_combined['runtime'] >= 150]
long_movie

In [ ]:
# short movie
short_movie = df_combined[df_combined['runtime'] <= 90]
short_movie

In [ ]:
long_revenue = long_movie['revenue']
short_revenue = short_movie['revenue']

#### Outliers

In [ ]:
# long revenue
zscores_long = stats.zscore(long_revenue)
outliers_long = abs(zscores_long)>3
print(f'Len: {len(long_revenue)}')
print(f'Number of Ouliers: {np.sum(outliers_long)}')

long_revenue = long_revenue[~outliers_long]
print(f'Len: {len(long_revenue)}')

In [ ]:
# short revenue
zscores_short = stats.zscore(short_revenue)
outliers_short = abs(zscores_short)>3
print(f'Len: {len(short_revenue)}')
print(f'Number of Ouliers: {np.sum(outliers_short)}')

short_revenue = short_revenue[~outliers_short]
print(f'Len: {len(short_revenue)}')

#### Normality
- Since both groups have sample count > 15, can skip this step entirely

#### Equal Variance
- Equal Variance assumption is not met
- Will add equal_var = False to test

In [ ]:
stats.levene(long_revenue, short_revenue)

### STEP 4: Perform Test & Interpret Result
- p value < .05, reject null hypothesis and support alternative hypothesis


- Runtime length does not affect revenue

In [ ]:
result_q2 = stats.ttest_ind(long_revenue, short_revenue, equal_var=False)
result_q2

In [ ]:
## is our result significant
print(f"p-value={result_q2.pvalue:.10f}")
print(f"Significant: {result_q2.pvalue <.05}")

### Visualization

In [ ]:
long_df = pd.DataFrame(long)
long_df['runtime'] = 'long'
long_df

In [ ]:
short_df = pd.DataFrame(short)
short_df['runtime'] = 'short'
short_df

In [ ]:
plot_q2 = pd.concat([long_df, short_df])
plot_q2

In [ ]:
print(f'Long Movie Revenue: {long.mean():,.2f}')
print(f'Short Movie Revenue: {short.mean():,.2f}')

In [ ]:
fig, ax = plt.subplots()

ax = sns.barplot(data=plot_q2, x='runtime', y='revenue')
ax.set_title('Runtime by Revenue');